# Разметка текстов сборов на адресные и неадресные по списку имен

### Проблема

Ранее с помощью библиотеки DeepPavlov из текстов датасета были извлечены (предполагаемые) имена людей и животных, а в датасет добавлена отдельная колонка со списком (для каждого текста) извлеченных имен или пустым списком, если имен не нашлось. Это было сделано для того, чтобы дальше определить, является ли сбор адресным или нет. Под адресными подразумеваются сборы на помощь конкретному человеку или животному. На краудфандинговой площадке все сборы являются целевыми, но не все адресными.

Но просто по наличию имен в тексте оказалось невозможно однозначно сказать, является ли сбор адресным или нет. Во-первых, модель часто ошибается и считает именами людей, например, названия организаций или лекарств. Во-вторых, даже при правильном определении имена могут быть случайными: имена волонтеров, врачей, сотрудников благотворительных организаций и так далее. 

Как правило, в текстах адресных сборов одно и то же имя (адресата сбора), упоминается несколько раз. Можно было бы предположить, что если одно и то же имя употребляется более n раз, то сбор является адресным. Но возникает проблема: как определить количество упоминаний, учитывая что одно имя может использоваться в разных словоизменительных и словообразовательных формах: Николай, Коля, Коленька… 

### Решение
Если проблемы словоизменения может решить лемматизация, то с разными словоформами имен не справляется ни лемматизация, ни стемминг. Убедившись в этом на практике, я предположила, что решить эту проблему можно с помощью **сравнения по расстоянию Левенштейна**. Можно сказать, что задача была решена достаточно успешно. Ниже приводится скрипт с использованием библиотеки Fuzzy Wuzzy. 

**FuzzyWuzzy** это библиотека для Python, которая используется для сопоставления строк. В основе лежит метрика - Расстояние Левенштейна. Библиотека используется, в частности, для определения близости смысла разных предложений в чат-ботах. Мы будем сравнивать не предложения, а слова. 

In [8]:
!pip install "fuzzywuzzy"
!pip install "python-Levenshtein"

     |████████████████████████████████| 48 kB 498 kB/s eta 0:00:01
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp37-cp37m-macosx_10_9_x86_64.whl size=79247 sha256=64d83a1a4bc84f3790ef9e67462a6e91014a1636df141bd26bc616411c541515
  Stored in directory: /Users/liza/Library/Caches/pip/wheels/f0/9b/13/49c281164c37be18343230d3cd0fca29efb23a493351db0009
Successfully built python-Levenshtein


In [9]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [184]:
import ast # The ast module helps Python applications to process trees of the Python abstract syntax grammar

\
У библиотеки fuzzywuzzy есть разные методы. [Краткая статья с примерами.](https://www.geeksforgeeks.org/fuzzywuzzy-python-library/) Сравним их на нашей задаче.⬇︎

In [389]:
# Списки для тестов
# Примечание: не стоит называть список просто list, это убьет (перепишет) втроенную функцию list!

list1 = ['Наташа','Дима', 'Наташи', 'Наталья','Дима', 'Димы', 'Дима', 'Дима', 'Димы', 'Лиза']
list2 = ['Ильнар', 'Кыш', 'Деда', 'Деда', 'Кыш', 'Снегурочки', 'Кар', 'Яна', 'Шурале', 'Шайтан', 'Убырлы', 'Баба', 'Аждаха', 'Змей', 'Батыр', 'Алтынчеч', 'Златовласка', 'Тахир', 'Зухра', 'Ромео', 'Деда', 'Кыш', 'Кыш', 'Кар', 'Дед', 'Кыш', 'Деда', 'Снегурочки']
list3 = ['Надежда', 'Островка', 'Островка']
list4 = ['Наташа', 'Даша']
list5 = sorted(list1) # сортирует по алфавиту

In [375]:
# Сравним разные методы
# На совсем разных именах:

print(fuzz.ratio('Дмитрий', 'Огабек'))
print(fuzz.token_sort_ratio('Дмитрий', 'Огабек'))
print(fuzz.partial_ratio('Дмитрий', 'Огабек')) # ищет по наименьшей строке

0
0
0


In [380]:
# На разных формах одного имени:

print(fuzz.ratio('Дмитрий', 'Дима'))
print(fuzz.token_sort_ratio('Дмитрий', 'Дима'))
print(fuzz.partial_ratio('Дмитрий', 'Дима'))

36
36
50


\
Методы fuzzy принимают только строки, не список. Поэтому нужно перебирать имена в цикле и сравнивать попарно. На выходе мы получим список коэффициентов - и посчитаем среднее для каждого. Проверим разные методы на одном и том же списке list1. ⬇︎ 


In [384]:
# здесь будет ошибка, т.к. подали список

print(fuzz.ratio(list1))
print(fuzz.token_sort_ratio(list1))
print(fuzz.partial_ratio(list1))

IndexError: tuple index out of range

In [386]:
# fuzz.ratio

a = []
#i=0
for i in range(len(list1)-1): # без вычитания единицы с конца в последней паре будет ошибка out of range
    a.append(fuzz.ratio(list1[i], list1[i+1])) # берем первое слово, сравниваем со следующим по порядку, потом его - со следующим за ним и т.д.
    #i = i+1
print(a) #список коэффициентов попарного сравнения
print(sum(a)/len(a)) # считаем среднее

[20, 20, 62, 18, 75, 75, 100, 75, 25]
52.22222222222222


In [398]:
# fuzz.partial_ratio

a = []
#i=0
for i in range(len(list1)-1):
    a.append(fuzz.partial_ratio(list1[i], list1[i+1]))
    #i = i+1
print(a)
print(sum(a)/len(a))

[25, 33, 67, 25, 75, 75, 100, 75, 25]
55.55555555555556


In [388]:
# fuzz.token_sort_ratio
# Результат такой же, как у просто ratio, что логично, т.к. у нас не предложения, а слова, поэтому сортировать в строке нечего. 

a = []
#i=0
for i in range(len(list1)-1):
    a.append(fuzz.token_sort_ratio(list1[i], list1[i+1]))
    #i = i+1
print(a)
print(sum(a)/len(a))

[20, 20, 62, 18, 75, 75, 100, 75, 25]
52.22222222222222


In [390]:
# сортированный по алфавиту список list5 = sorted(list1)
# ratio
a = []
#i=0
for i in range(len(list5)-1):
    a.append(fuzz.ratio(list5[i], list5[i+1]))
    #i = i+1
print(a)
print(sum(a)/len(a))

[100, 100, 100, 75, 100, 25, 18, 62, 83]
73.66666666666667


In [391]:
# сортированный по алфавиту список list5 = sorted(list1)
# partial_ratio

a = []
i=0
for i in range(len(list5)-1):
    a.append(fuzz.partial_ratio(list5[i], list5[i+1]))
    i = i+1
print(a)
print(sum(a)/len(a))

[100, 100, 100, 75, 100, 25, 25, 67, 83]
75.0


### Выбор метода библиотеки

Видно, что результаты разных методов существенно отличаются. В нашем тестовом примере список имен скорее похож на адресный (имя Дима неслучайно повторяется так много раз). Поэтому может показаться, что чем выше для него коэффициент схожести имен, тем лучше подходит метод. 

Но на коэфф. могут повлиять случайные факторы, например: 
* Случайная созвучность разных имен (Лиза, Лида, Лика; Даша, Маша, Паша, Саша, Наташа vs Саша, Александр)
* Количество имен имен в тексте (кто-то из авторов чаще, чем другие, может использовать местоимения, избегая повторов, а где-то встречается целый список артистов - участников концерта).
* Порядок их упоминания в тексте (т.к. мы сравниваем имена попарно, если в несколько упоминаний имени Коля влезло упоминание, скажем, мамы Коли Наташи, это понизит “вес” имени адресата сбора (“Коля”), а если, по воле автора текста, имена будут плотнее сгруппированы (все про Колю в начале, а в конце комментарии мамы - повысит общий коэффициент).

Поскольку мы не знаем распределение этих (и других возможных) факторов в датасете и ко всему датасету применяем один и тот же метод, выбрать идеальный не представляется возможным. Дальше я использую простой ratio.

## Запускаем метод на датасете

In [346]:
import pandas as pd
import numpy as np

In [359]:
def fuzzy_mean(alist):
    #alist = sorted(alist) # сортировка списка в разных случаях приносит разный эффект, в общей массе на результат сильно не влияет
    if len(alist) > 1: #если в списке больше одного имени, иначе нет смысла сравнивать и будет ошибка: деление на 0 (len()-1 = 0)
        a = []
        i=0
        for i in range(len(alist)-1):
            a.append(fuzz.ratio(alist[i], alist[i+1]))
            i = i+1
        #print(a)
        mean = round(sum(a)/len(a)) # среднее с округлением
        if mean > 50: # этот порог можно менять, значения адр\неадресный пока не пишутся в датасет, это в другой функции
            print(alist) # смотрим на имена, чья схожесть оказалась высокой
            print("similarity rate:", mean) # и на среднее значение
            print("\n")
    else: # если в списке меньше одного имени, ничего не считай, просто записывай 0 
        mean = 0    
    return(round(mean))

In [357]:
# эта функция создает в датасете новую колонку, куда пишет полученное среднее схожести имен

def df_upd(df):
    df = df.assign(NamesSimRate = SIMILARITY)
    return df

In [358]:
# т.к. ранее при извлечении имен в датасет также записывалась разметка адресный\неадресный, этой функцией мы переписываем значения колонки на более точные

def adr_refresh(df):
    bins = [-np.inf, 50, np.inf] # 50 это значение в колонке NamesSimRate которое становится условием (границей) разметки 
                                # этот порог можно менять, но похоже, что на этом датасете такая граница оптимальная
    labels = ['False','True'] # True если сбор адресный, False если нет. 
    df['Adr'] = pd.cut(df['NamesSimRate'], bins=bins, labels=labels) # pandas.cut allows to segment and sort data values into bins
    return(df)

In [373]:
%%time 

# %%time показывает время работы ячейки

if __name__ == "__main__": # в Jupyter необязательная строка
    
    mydf = pd.read_csv('/Users/liza/PycharmProjects/Planeta_project/plset_fin_upd_adr.csv')
    SIMILARITY = [] # станет колонкой датасета с ретингом схожести имен
    for names_list in mydf['Names']: # в этой колонки списки имен, извлеченных раньше
        names_list = list(ast.literal_eval(names_list)) 
        # при записи датасета имена были списком, но из файла читаются как строка. list(ast.literal_eval(names_list)) позволяет прочитать их как списки и итерироваться по ним для подсчета схожести
        
        SIMILARITY.append(fuzzy_mean(names_list))
    mydf = df_upd(mydf) # пишем в датасет рейтинг схожести имен
    mydf = adr_refresh(mydf) # переписываем колонкку с разметкой на адресные и неадресные сборы
    mydf = mydf.drop(mydf.columns[0:2], axis=1) # удаляем старые колонки с индексами, которые в pandas появляются при каждом чтении\записи датасета
    mydf.to_csv("plset_ver_010.csv") # сохраняем новый датасет в файл

['Надежда', 'Островка', 'Островка']
similarity rate: 56


['Сергею', 'Сергей', 'Сергей', 'Сергей', 'Сергей', 'Сергей', 'Сергей', 'Сергея', 'Сергея', 'Сергей', 'Сергея', 'Сергея']
similarity rate: 94


['Вадима', 'Вадюшка', 'Вадику', 'Вадику', 'Вадик', 'Вадима', 'Вадиму']
similarity rate: 78


['Алиса', 'Рентген', 'Лаки', 'Лаки', 'Лаки', 'Лаки', 'Бумеранга']
similarity rate: 52


['Диана', 'Диана', 'Дианочка', 'Дианы', 'Пауль', 'Диана', 'Дианы']
similarity rate: 60


['Марасакин', 'Марк', 'Марка', 'Марк', 'Марка', 'Марка', 'Марка', 'Марка', 'Марк', 'Марк', 'Марк', 'Марка', 'Марк', 'Марк', 'Марк', 'Марку', 'Марка', 'Марку']
similarity rate: 91


['Ани', 'Ани']
similarity rate: 100


['Алеши', 'Алеша', 'Алеша', 'Алеше', 'Лёше', 'Алеши']
similarity rate: 65


['Арсению', 'Арсению', 'Гармонь', 'Арсений', 'Арсения', 'Арсений', 'Арсения', 'Арсению', 'Арсений', 'Арсений', 'Арсений', 'Арсений', 'Арсений', 'Арсений', 'Арсения', 'Сыроватский', 'Арсений', 'Арсению', 'Арсению', 'Арсения', 'Арсения'

In [371]:
mydf # проверяем: все записалось

,Title,Author,Description,Briefly,Goal,Result,Donations,Start,Finish,LengthZn,...,Started,Finished,LeadBag,DaysLong,Cluster,Category,Rate,Names,Adr,NamesSimRate
0,Подарим жизнь,Бобруйское общественное объединение защиты жив...,"Друзья, рады приветствовать на странице нашего...",Спасение животных от эвтаназии на 5 сутки посл...,10000.0,76100,67,11 сентября 2017,9 ноября 2017,1134,...,11 Sep 2017,9 Nov 2017,"['спасение', 'животное', 'эвтаназия', 'сутки',...",59,7,животные,1290.0,['Эгиды'],False,0
1,Оплата нянечек для отказных детей в больницах,"Благотворительный фонд ""Дети без мам""","Когда ребёнок серьёзно болен, он ложится в бол...",Сбор средств на оплату ухода за детьми-сиротам...,150000.0,711665,470,1 июля 2014,16 сентября 2014,1749,...,1 Jul 2014,16 Sep 2014,"['сбор', 'средство', 'оплата', 'уход', 'ребёно...",77,0,сироты_дети_из_неблагополучных семей,9240.0,[],False,0
2,Оплата нянь для детей-отказников в больницах,"Благотворительный фонд ""Дети без мам""","В больницах живут малыши, оставшиеся без попеч...",Сбор средств на уход за детьми-отказниками в б...,50000.0,236257,202,14 октября 2013,16 января 2014,1693,...,14 Oct 2013,16 Jan 2014,"['сбор', 'средство', 'уход', 'ребёнок', 'отказ...",94,0,сироты_дети_из_неблагополучных семей,2510.0,"['Зою', 'Ксану', 'Анну', 'Юрия', 'Фёдора', 'Ал...",False,11
3,"""Не вешай нос! Или право на жизнь""",Местная общественная организация защитников жи...,"И снова здравствуйте, Друзья! \nМеня зовут Над...",Цель проекта - ликвидировать долги за ветерина...,29395.0,95300,170,21 февраля 2017,21 апреля 2017,2138,...,21 Feb 2017,21 Apr 2017,"['цель', 'проект', 'ликвидировать', 'долг', 'в...",59,7,животные,1620.0,"['Надежда', 'Кипиш', 'Островка', 'Маня', 'Хрюн...",False,35
4,Календарь реабилитационного центра для птиц,Вера Пахомова,"Привет, друг! Мы снова на Planeta.ru!) \n В н...","""Воронье Гнездо"" собирает средства на печать к...",15000.0,42550,87,9 ноября 2019,8 декабря 2019,1765,...,9 Nov 2019,8 Dec 2019,"['воронья', 'гнездо', 'собирать', 'средство', ...",29,4,активизм_просвещение_профилактика,1470.0,[],False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2033,Поможем вместе двойняшкам Артемию и Арине!,БФ Милосердие,Благотворительный фонд помощи детям и инвалида...,7 декабря малышам исполнится 5 лет. Давайте сд...,120268.0,0,0,17 октября 2019,10 января 2020,1952,...,17 Oct 2019,10 Jan 2020,"['декабрь', 'малыш', 'исполниться', 'год', 'да...",85,1,дети_лечение_реабилитация,0.0,"['Арине', 'Теме', 'Наталья', 'Арина', 'Артемий...",False,42
2034,Пункт кормления,СРОО Пункт Милосердия,Здравствуйте! Меня зовут Снежана! Мне 35 лет. ...,"В ""Пункте кормления"" готовятся и БЕСПЛАТНО раз...",333600.0,421,3,26 февраля 2020,31 марта 2020,4016,...,26 Feb 2020,31 Mar 2020,"['пункт', 'кормление', 'готовиться', 'бесплатн...",34,4,бездомные_кризис,10.0,['Снежана'],False,0
2035,"SMG & Bees AWARDS 2020 Москва,Vegas City Hall","Благотворительный фонд ""АННА""",Друзья!\nУже в третий раз мы проводим междунар...,Bees Media Group проводит церемонию вручений п...,300000.0,0,0,22 декабря 2019,18 января 2020,1453,...,22 Dec 2019,18 Jan 2020,"['проводить', 'церемония', 'вручение', 'премия...",27,4,активизм_просвещение_профилактика,0.0,"['Авраам', 'Татьяна', 'Наташа', 'Бьянка', 'Нар...",False,24
2036,Город мастеров: швейный цех 2.0,"АНО ""Открытый город""","Знакомьтесь, это мы - ""Город мастеров"" - инклю...","""Город мастеров"" - инклюзивные мастерские из Е...",200000.0,451,4,6 февраля 2020,15 апреля 2020,1734,...,6 Feb 2020,15 Apr 2020,"['город', 'мастер', 'инклюзивный', 'мастерский...",69,12,социализация_возможности,10.0,[],False,0


In [392]:
mydf[mydf['Adr']=='True'] # все верно, если NamesSimRate >50, то в Adr - True 

,Title,Author,Description,Briefly,Goal,Result,Donations,Start,Finish,LengthZn,...,Started,Finished,LeadBag,DaysLong,Cluster,Category,Rate,Names,Adr,NamesSimRate
6,Построим вместе! Печь для приготовления каши,Местная общественная организация защитников жи...,"Меня зовут Надежда Николаевна, я директор Мест...","Центр реабилитации для собак ""Островок надежды...",46365.0,104500,217,15 ноября 2016,15 декабря 2016,2548,...,15 Nov 2016,15 Dec 2016,"['центр', 'реабилитация', 'собака', 'островок'...",30,3,животные,3480.0,"['Надежда', 'Островка', 'Островка']",True,56
14,Защитим Кислухинский заказник от браконьеров!,"МОО ""Экологический клуб""","Здравствуйте, друзья!\nМы приглашаем вас к уча...","Кислухинский лес, его обитатели и егерь Сергей...",380000.0,654200,936,16 декабря 2015,10 февраля 2016,4318,...,16 Dec 2015,10 Feb 2016,"['кислухинский', 'лес', 'обитатель', 'егерь', ...",56,4,экология,11680.0,"['Сергею', 'Сергей', 'Сергей', 'Сергей', 'Серг...",True,94
15,Помочь Вадиму встать на ноги!,"Благотворительный фонд ""Золотце""","Добрый, день друзья!\nНаш Благотворительный фо...",Реабилитационный курс для мальчика с детский ц...,16500.0,28165,52,18 февраля 2017,10 марта 2017,1518,...,18 Feb 2017,10 Mar 2017,"['реабилитационный', 'курс', 'мальчик', 'детск...",20,1,дети_лечение_реабилитация,1410.0,"['Вадима', 'Вадюшка', 'Вадику', 'Вадику', 'Вад...",True,78
23,Сбор средств на коляску для пса Лаки,Бумеранг Добра,"Здравствуйте, меня зовут Алиса Корецкая ,я вол...",Собаке-инвалиду Лаки после огнестрельного ране...,16000.0,24250,32,20 декабря 2016,31 декабря 2016,1410,...,20 Dec 2016,31 Dec 2016,"['собака', 'инвалид', 'лак', 'огнестрельный', ...",11,7,животные,2200.0,"['Алиса', 'Рентген', 'Лаки', 'Лаки', 'Лаки', '...",True,52
26,"Нежная ""бабочка"" Диана","Благотворительный Фонд ""Дети-бабочки""",Двухлетняя Диана из Сургута - ребенок-бабочка....,Диана - девочка-бабочка. От рождения ее кожа о...,180000.0,264772,247,6 июля 2015,4 октября 2015,1635,...,6 Jul 2015,4 Oct 2015,"['диана', 'девочка', 'бабочка', 'рождение', 'к...",90,11,дети_лечение_реабилитация,2940.0,"['Диана', 'Диана', 'Дианочка', 'Дианы', 'Пауль...",True,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2003,Дети-ангелы среди нас!,"БФ ""Подари ЗАВТРА!""",Благотворительный фонд «Подари ЗАВТРА!» был ос...,"Дети-ангелы - это искренние, добрые и любящие ...",450000.0,500,4,19 сентября 2018,10 декабря 2018,1741,...,19 Sep 2018,10 Dec 2018,"['ребёнок', 'ангел', 'искренний', 'добрый', 'л...",82,1,дети_лечение_реабилитация,10.0,"['Адели', 'Марат', 'Марату', 'Маратик', 'Марат...",True,55
2004,Поможем Арсену начать ходить,Свердловская региональная общественная некомме...,"Арсену 7 лет, он родился недоношенным. \nВо вр...",Сбор средств на приобретение ходунков для мале...,103000.0,0,0,27 сентября 2018,30 ноября 2018,1399,...,27 Sep 2018,30 Nov 2018,"['сбор', 'средство', 'приобретение', 'ходунок'...",64,1,дети_лечение_реабилитация,0.0,"['Арсену', 'Арсен', 'Чуковского', 'Арсен', 'Ар...",True,54
2020,Волшебный костюм для Саввы,"БФ ""Подари ЗАВТРА!""",Благотворительный фонд «Подари ЗАВТРА!» был ос...,"Благотворительный фонд ""Подари ЗАВТРА!"" призыв...",38700.0,0,0,13 марта 2019,15 марта 2019,1721,...,13 Mar 2019,15 Mar 2019,"['благотворительный', 'фонд', 'подарить', 'зав...",2,1,дети_лечение_реабилитация,0.0,"['Савелий', 'Савве', 'Саввы', 'Юлия', 'Савелия...",True,58
2027,Как помочь детям-ангелам?,"БФ ""Подари ЗАВТРА!""",Благотворительный фонд «Подари ЗАВТРА!» был ос...,Помочь детям-ангелам очень просто – принять уч...,100000.0,0,0,24 июня 2019,29 июля 2019,2101,...,24 Jun 2019,29 Jul 2019,"['помочь', 'ребёнок', 'ангел', 'очень', 'прост...",35,1,дети_лечение_реабилитация,0.0,"['Адели', 'Вася', 'Вася', 'Васенька', 'Марат',...",True,69


In [372]:
# del mydf (очистка переменной при необходимости)

# Итог
Результат сравнения по расстоянию Левенштена выглядит довольно чистым: списки имен с относительно высоким рейтингом действительно похожи на адресные сборы. Лишние тексты тоже попали в выборку (Например, "Надежда, Отстровок, Островок"), но для дальнейшего исследования это можно поправить в датасете вручную. В любом случае такой подход упрощает задачу такой разметки. 

Вероятно, улучшить фильтрацию можно с помощью предвариетельной лемматизации. 